# IMDB 영화리뷰 감성분석 - Conv1D
- Conv1D + Conv1D + Dense
- Conv1D + LSTM + Dense
- Conv1D + Dense + Dense

In [4]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [6]:
num_words = 10000
(X_train, y_train), (_, _) = imdb.load_data(num_words=num_words)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [8]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

- Case 1) Conv1D x 2

In [9]:
model1 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [10]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model1_path = 'best-imdb-conv1d-conv1d.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [11]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc1,es1]
)

Epoch 1/30
249/250 [============================>.] - ETA: 0s - loss: 0.5576 - accuracy: 0.6770
Epoch 1: val_loss improved from inf to 0.36056, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 17s 10ms/step - loss: 0.5569 - accuracy: 0.6777 - val_loss: 0.3606 - val_accuracy: 0.8465
Epoch 2/30
245/250 [============================>.] - ETA: 0s - loss: 0.2885 - accuracy: 0.8822
Epoch 2: val_loss improved from 0.36056 to 0.32152, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2891 - accuracy: 0.8819 - val_loss: 0.3215 - val_accuracy: 0.8630
Epoch 3/30
245/250 [============================>.] - ETA: 0s - loss: 0.1945 - accuracy: 0.9259
Epoch 3: val_loss did not improve from 0.32152
250/250 [==============================] - 3s 10ms/step - loss: 0.1947 - accuracy: 0.9258 - val_loss: 0.3492 - val_accuracy: 0.8570
Epoch 4/30
248/250 [============================>.] - ETA: 0s - loss: 0.1320 - accu

In [12]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.3084 - accuracy: 0.8682


[0.30842888355255127, 0.8682000041007996]

- Case 2) Conv1D + LSTM

In [13]:
model2 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [14]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model2_path = 'best-imdb-conv1d-lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [15]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2,es2]
)

Epoch 1/30
247/250 [============================>.] - ETA: 0s - loss: 0.4429 - accuracy: 0.7661
Epoch 1: val_loss improved from inf to 0.29108, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 6s 16ms/step - loss: 0.4409 - accuracy: 0.7677 - val_loss: 0.2911 - val_accuracy: 0.8848
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2170 - accuracy: 0.9147
Epoch 2: val_loss improved from 0.29108 to 0.26511, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 3s 13ms/step - loss: 0.2170 - accuracy: 0.9147 - val_loss: 0.2651 - val_accuracy: 0.8905
Epoch 3/30
250/250 [==============================] - ETA: 0s - loss: 0.1442 - accuracy: 0.9481
Epoch 3: val_loss did not improve from 0.26511
250/250 [==============================] - 3s 14ms/step - loss: 0.1442 - accuracy: 0.9481 - val_loss: 0.2915 - val_accuracy: 0.8857
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.0996 - accuracy

In [16]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.2721 - accuracy: 0.8884


[0.272092342376709, 0.8884000182151794]

- Case 3) Conv1D + Dense

In [17]:
model3 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [18]:
model3 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_3 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 100)              

In [19]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model3_path = 'best-imdb-conv1d-dense.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [20]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc3,es3]
)

Epoch 1/30
244/250 [============================>.] - ETA: 0s - loss: 0.5707 - accuracy: 0.6772
Epoch 1: val_loss improved from inf to 0.39085, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 3s 10ms/step - loss: 0.5664 - accuracy: 0.6808 - val_loss: 0.3909 - val_accuracy: 0.8240
Epoch 2/30
245/250 [============================>.] - ETA: 0s - loss: 0.3116 - accuracy: 0.8678
Epoch 2: val_loss improved from 0.39085 to 0.30105, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.3119 - accuracy: 0.8679 - val_loss: 0.3010 - val_accuracy: 0.8767
Epoch 3/30
246/250 [============================>.] - ETA: 0s - loss: 0.2062 - accuracy: 0.9186
Epoch 3: val_loss improved from 0.30105 to 0.28300, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2063 - accuracy: 0.9186 - val_loss: 0.2830 - val_accuracy: 0.8835
Epoch 4/30
245/250 [====================

In [21]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.2849 - accuracy: 0.8836


[0.28485625982284546, 0.8835999965667725]